## https://github.com/tiagomartins-threesigma/SINFO2023FRI4STARKs

In [1]:
%run Base.ipynb

prime = 12289


# FRI Query Phase
In the query phase, the verifier tries to find inconsistencies between the previously committed sequence of polynomial evaluations and the new evaluations that are computed from the queries. 

### Deserialize the FRI Commitment
First, to simulate the existence of an oracle we will import the data from the FRI Commitment file. In reality, commitments are implemented by building Merkle trees, and decommitments are accompanied with Merkle-proofs. But here, we will just "import" the lists $\texttt{p}$ and $\texttt{z}$ from an external file through Pickle object deserialization.

Moreover, the polynomial of the last layer must be constant, so we will use one of its evaluatio values to define the constant $C$. 

In [2]:
import pickle
file = open('FRI_Commitment_File.out', 'rb')
data = pickle.load(file)
file.close()

p = data['p']
z = data['z']
C = p[-1][0]

At the last layer, $i=\mathrm{LL}$ which expresses how many times $d_0$ needs to be divided by $2$ in order to have $d_\mathrm{LL}\le 1$. Hence, $\mathrm{LL} = \mathrm{ceil} (\log_2 (d_0))$.


Next, we define the group generator $g$ that is used to express the $x$'s. One has $x_k = g^k$ which is to say that, at position $\texttt{sample}$ one has $\texttt{x = gen**sample}$.

Recall the group cyclic structure:  


<div>
<img src="group_structure.png" width="300"/>
</div>


In [7]:
LL = int(np.ceil(np.log2(d0)))
gen = Fp.primitive_element**((prime-1)//len(p[0]))

print('Last layer i =', LL)
print('gen =', gen)

Last layer i = 4
g = 7311


### FRI Query Round

1. Compute the $x$ corresponding to the position $\texttt{sample}$ on layer $0$
2. Retrieve $p_0(x)$ from the commitment of layer $0$ at position $\texttt{sample}$
3. Cycle through all layers $0< i < \mathrm{LL}$ in sequence and perform the operations:

    1. Define half of the length of the layer which is useful to locate $-x$ at position $\texttt{sample-halflength}$
    2. Retrieve $p_i(x)$ from the commitment of layer $i$ at position $\texttt{sample-halflength}$
    3. Compute $g(x^2) = (p_i(x) + p_i(-x))/2$
    4. Compute $h(x^2) = (p_i(x) - p_i(-x))/(2x)$
    5. Compute $p_{i+1}(x^2) = z_{i,0} \, g(x^2) + z_{i,1} \, h(x^2)$
    6. Retrieve $p_{i+1}(x^2)$ from the commitment of layer $i+1$ at position $\texttt{sample%halflength}$
    7. Verify whether there is a match for $p_{i+1}(x^2)$
    8. Set $x \gets x^2$

4. Test whether the last layer polynomial is constant: $p_{\mathrm{LL}}(x) == C$.

In [20]:
def verify(sample: int):
    x = gen**sample
    pplus = p[0][sample]
    
    # check consistency between layers
    for i in range(LL):
        half_length = len(p[i])//2
        pminus = p[i][sample-half_length]
        g = (pplus + pminus)/Fp(2)
        h = (pplus - pminus)/(Fp(2)*x)
        
        pplus = z[i][0]*g + z[i][1]*h
        sample = sample % half_length
        assert pplus == p[i+1][sample], f'At position {sample}, layer {i+1} is not consistent with layer {i}.'
        x = x**2
        
    # check that the last layer is equal to the specified constant
    assert pplus == C, f'At position {sample}, the last layer "Layer {i+1}" does not equal the constant {C}.'
    
    print(f'For this specific sample, the FRI layers were consistent.')
    

### Let us Verify!

verify(4)

### What if the verifier makes multiple random samples?

In [23]:
samples = set([rand(d0*BUF, dtype=int) for i in range(6)])
print('samples =', samples, end='\n \n')

for sample in samples:
    verify(sample)
    print(f'Verified -> {sample}')

samples = {35, 5, 38, 11, 15, 47}
 


AssertionError: At position 3, the last layer "Layer 4" does not equal the constant 9698.